In [0]:
%sql
-- init of Unity Catalog, using external location for pipeline data
CREATE CATALOG IF NOT EXISTS gemeente
MANAGED LOCATION 'abfss://uc-catalog@storagegemeente.dfs.core.windows.net/';

CREATE SCHEMA IF NOT EXISTS gemeente.problem;
CREATE SCHEMA IF NOT EXISTS gemeente.lookup;

CREATE SCHEMA IF NOT EXISTS gemeente.bronze;
CREATE SCHEMA IF NOT EXISTS gemeente.silver;
CREATE SCHEMA IF NOT EXISTS gemeente.gold;

CREATE TABLE IF NOT EXISTS gemeente.lookup.coords;

CREATE TABLE IF NOT EXISTS gemeente.bronze.reports
USING DELTA
LOCATION 'abfss://bronze@storagegemeente.dfs.core.windows.net/reports';

CREATE TABLE IF NOT EXISTS gemeente.silver.reports
USING DELTA
LOCATION 'abfss://silver@storagegemeente.dfs.core.windows.net/reports';

CREATE TABLE IF NOT EXISTS gemeente.gold.reports
USING DELTA
LOCATION 'abfss://gold@storagegemeente.dfs.core.windows.net/reports';

CREATE TABLE IF NOT EXISTS gemeente.problem.problem_id_tracker (
    last_id LONG
);

INSERT INTO gemeente.problem.problem_id_tracker VALUES ('problem_id', 0);

In [0]:
# remove all report data from adls storage
layers = ['source', 'bronze', 'silver', 'gold']
reports_adls_path = {layer: f'abfss://{layer}@storagegemeente.dfs.core.windows.net/reports' for layer in layers}

for layer, path in reports_adls_path.items():
    if dbutils.fs.ls(path):
        print(f'Removing {layer} layer at {path}')
        dbutils.fs.rm(path, recurse=True)

In [0]:
%sql
-- drop all delta tables to remove schema expectations for full new clean run
DROP TABLE IF EXISTS gemeente.bronze.reports;
DROP TABLE IF EXISTS gemeente.silver.reports;
DROP TABLE IF EXISTS gemeente.gold.reports;